Prerequisite installs for Project 3 - Quantum key distribution with entangled qubits (Simulation)

In [ ]:
%pip install qiskit
%pip install qiskit[visualization]
%pip install qiskit-aer

Project 3 - Quantum key distribution with entangled qubits (Simulation)

Step 1

In [ ]:
from qiskit               import QuantumCircuit, \
                                 transpile
from qiskit_aer           import AerSimulator
from qiskit.visualization import plot_histogram


qc = QuantumCircuit(2, 2)
qc.x(1)     # Flip Bob's qubit to |1>
qc.h(0)     # Create superposition on Alice's qubit
qc.cx(0, 1) # Entangle Alice and Bob's qubits
qc.z(0)     # Apply Z-gate to Alice's qubit to get |ψ_ab>
qc.measure([0, 1], [0, 1]) # Measure both qubits

print(qc)

backend = AerSimulator() # Initialize backend for simulation
transpiled_qc = transpile(qc, backend)

# Run the transpiled circuit on the backend
job = backend.run(transpiled_qc, shots=1000)
result = job.result()
counts = result.get_counts()

print(f"The counts for the circuit are: {counts}")
plot_histogram(counts)

Step 2

In [ ]:
from qiskit     import QuantumCircuit, \
                       transpile
from qiskit_aer import AerSimulator
from math       import radians


def create_psi_ab_circuit_with_angle(angle_a, angle_b):
    qc = QuantumCircuit(2, 2)
    qc.reset([0, 1]) # Reset qubits to |0⟩
    qc.x(1)          # Flip Bob's qubit to |1>
    qc.h(0)          # Create superposition on Alice's qubit
    qc.cx(0, 1)      # Entangle Alice and Bob's qubits
    qc.z(0)          # Apply Z-gate to Alice's qubit to get |ψ_ab>

    # Convert angles to radians and apply rotations
    angle_a_rad = radians(angle_a)
    angle_b_rad = radians(angle_b)
    qc.ry(2 * angle_a_rad, 0) # Rotate Alice's qubit
    qc.ry(2 * angle_b_rad, 1) # Rotate Bob's qubit

    # Add measurement
    qc.measure([0, 1], [0, 1])

    return qc

# Define the four circuits for the specified orientations
qc_00_00 = create_psi_ab_circuit_with_angle(0, 0)
print("Circuit (0°, 0°):")
print(qc_00_00)

qc_minus30_00 = create_psi_ab_circuit_with_angle(-30, 0)
print("Circuit (-30°, 0°):")
print(qc_minus30_00)

qc_00_30 = create_psi_ab_circuit_with_angle(0, 30)
print("Circuit (0°, 30°):")
print(qc_00_30)

qc_minus30_30 = create_psi_ab_circuit_with_angle(-30, 30)
print("Circuit (-30°, 30°):")
print(qc_minus30_30)

backend = AerSimulator() # Initialize backend for simulation

# Run each circuit and display results
for qc, (angle_a, angle_b) in [
    (qc_00_00,      (  0,  0)),
    (qc_minus30_00, (-30,  0)),
    (qc_00_30,      (  0, 30)),
    (qc_minus30_30, (-30, 30))
]:
    # Transpile and execute
    transpiled_qc = transpile(qc, backend)
    shots = 1024
    job = backend.run(transpiled_qc, shots=shots)
    result = job.result()
    counts = result.get_counts()

    # Display the results
    counts_00 = counts.get('00', 0)
    counts_01 = counts.get('01', 0)
    counts_10 = counts.get('10', 0)
    counts_11 = counts.get('11', 0)
    print(f"Results for angles ({angle_a}°, {angle_b}°):\n00: {counts_00}, 01: {counts_01}, 10: {counts_10}, 11: {counts_11}")
    print(f"Probability of measuring |00⟩: {(counts_00) / shots:.3f}")
    print(f"Probability of measuring |11⟩: {(counts_11) / shots:.3f}")
    print()

Step 3

In [ ]:
from random import randint

def generate_random_settings(size=1024):
    """
    Generate a random string for measurement settings.
    Each setting is randomly chosen from [0, 1], representing:
    0 -> Measurement angle 0°
    1 -> Measurement angle -30° or 30°.
    """
    return [randint(0, 1) for _ in range(size)]

# Generate random settings for Alice and Bob
alice_settings = generate_random_settings(size=1024)
bob_settings   = generate_random_settings(size=1024)

# Print a preview of the settings
print(f"Alice's settings (first 20): {alice_settings[:20]}")
print(f"Bob's settings (first 20): {bob_settings[:20]}")

Step 4

In [ ]:
from qiskit     import transpile
from qiskit_aer import AerSimulator

# Function to perform random measurements
def run_random_measurements(alice_settings, bob_settings, simulator):
    results = {}
    
    for i in range(min(len(alice_settings), len(bob_settings))):
        angle_a = 0 if alice_settings[i] == 0 else -30
        angle_b = 0 if bob_settings[i] == 0 else 30
        
        qc = create_psi_ab_circuit_with_angle(angle_a, angle_b)
        transpiled_qc = transpile(qc, simulator)
        
        # Run simulation with multiple shots
        job = simulator.run(transpiled_qc, shots=1)
        result = job.result()
        counts = result.get_counts()
        
        # Calculate the most likely outcome
        max_outcome = max(counts, key=counts.get)
        results[i] = (max_outcome, counts)
    
    return results

# Instantiate the Aer simulator
simulator = AerSimulator()

# Run the measurement protocol
results = run_random_measurements(alice_settings, bob_settings, simulator)

# Print the outcomes and counts
for index, (outcome, counts) in results.items():
    print(f"Index {index}: Outcome={outcome}, Counts={counts}")

Step 5

In [ ]:
# Initialize the simulator
backend = AerSimulator()

# Generate the circuit for configuration (0°, 0°)
qc_00_00 = create_psi_ab_circuit_with_angle(0, 0)

# Transpile and execute the circuit
transpiled_qc = transpile(qc_00_00, backend)
job = backend.run(transpiled_qc, shots=1024)
result = job.result()
counts = result.get_counts()

key_list = []
for outcome, count in counts.items():
    key_list.extend([outcome] * count)  # Add the outcome 'count' times to the key list

# Print the key
print("Generated Key (Measurement Outcomes):")
print(key_list)